In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import sys
import logging
import inspect
logging.basicConfig(level=logging.INFO)
print(sys.executable)

/n/groups/datta/tim_sainburg/conda_envs/peromoseq/bin/python3


### Get recording info (google sheets)

In [3]:
import requests
import pandas as pd
from io import BytesIO

In [4]:
# spreadsheet_url = 'https://docs.google.com/spreadsheet/ccc?key=14HIqUaSl_n-91hpAvmACY_iVY9nLKdlA6qklhxfZon0&output=csv&gid=0'
spreadsheet_url = "https://docs.google.com/spreadsheet/ccc?key=1jACsUmxuJ9Une59qmvzZGc1qXezKhKzD1zho2sEfcrU&output=csv&gid=0"
response = requests.get(spreadsheet_url)
recording_df = pd.read_csv(BytesIO(response.content))

In [5]:
recording_df[:3]

,Subject,duration_m,video_recording_id,ephys_id,calibration_id,video_location_on_o2,ephys_location_on_o2,calibration_location_on_o2,samplerate,username,n_ephys_streams,max_video_duration_m
0,M04002,10,24-05-01-13-26-43-110846,2024-05-01_13-26-37,24-05-01-13-45-07-825493,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,150,tis697,1,10


### Submit job

In [6]:
from multicamera_airflow_pipeline.tim_240731.interface.o2 import O2Runner
from pathlib import Path
from datetime import datetime

INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Python interpreter binary location: /n/groups/datta/tim_sainburg/conda_envs/peromoseq/bin/python3
/n/groups/datta/tim_sainburg/conda_envs/peromoseq/lib/python3.10/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/n/groups/datta/tim_sainburg/conda_envs/peromoseq/lib/python3.10/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [7]:
for idx, recording_row in recording_df.iterrows():
    break

In [8]:
recording_row

Subject                                                                  M04002
duration_m                                                                   10
video_recording_id                                     24-05-01-13-26-43-110846
ephys_id                                                    2024-05-01_13-26-37
calibration_id                                         24-05-01-13-45-07-825493
video_location_on_o2          /n/groups/datta/tim_sainburg/datasets/chronic2...
ephys_location_on_o2          /n/groups/datta/tim_sainburg/datasets/chronic2...
calibration_location_on_o2    /n/groups/datta/tim_sainburg/datasets/chronic2...
samplerate                                                                  150
username                                                                 tis697
n_ephys_streams                                                               1
max_video_duration_m                                                         10
Name: 0, dtype: object

### Job specific

In [9]:
import yaml
import textwrap
import time

In [10]:
# these are arguments
output_directory = Path("/n/groups/datta/tim_sainburg/datasets/scratch/") / "240808-3d-pipeline"
job_directory = Path('/n/groups/datta/tim_sainburg/datasets/scratch/jobs')
job_directory.mkdir(exist_ok=True, parents=True)
config_file = Path("/n/groups/datta/tim_sainburg/projects/multicamera_airflow_pipeline/multicamera_airflow_pipeline/tim_240731/default_config.yaml")

In [11]:
def convert_minutes_to_hms(minutes_float):
    # Convert minutes to total seconds
    total_seconds = int(minutes_float * 60)
    
    # Extract hours, minutes, and seconds using divmod
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    
    # Format as HH:MM:SS
    return f"{hours:02}:{minutes:02}:{seconds:02}"

In [12]:
def check_2d_completion(output_directory_predictions):
    return (output_directory_predictions / "completed.log").exists()

In [13]:
# load config
config_file = Path(config_file)
config = yaml.safe_load(open(config_file, "r"))

In [14]:
# where the video data is located
recording_directory = (
    Path(recording_row.video_location_on_o2) / recording_row.video_recording_id
)

In [15]:
# where to save output
output_directory_predictions = (
    output_directory / "2D_predictions" / recording_row.video_recording_id
)
output_directory_predictions.mkdir(parents=True, exist_ok=True)
current_datetime_str = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
remote_job_directory = job_directory / current_datetime_str

# duration of video files (useful if videos are not properly muxxed)
expected_video_length_frames = recording_row.max_video_duration_m * recording_row.samplerate

# where tensorrt compiled models are saved (specific to GPU, so we compute on the fly)
tensorrt_model_directory = output_directory / "tensorrt"

params = {
    "recording_directory": recording_directory.as_posix(),
    "output_directory_predictions": output_directory_predictions.as_posix(),
    "expected_video_length_frames": expected_video_length_frames,
    "tensorrt_model_directory": tensorrt_model_directory.as_posix(),
}

duration_requested = convert_minutes_to_hms(
    recording_row.duration_m * config["o2"]["prediction_2d"]["o2_runtime_multiplier"]
)

In [16]:
# create the job runner
runner = O2Runner(
    job_name_prefix=f"{recording_row.video_recording_id}_2d_predictions",
    remote_job_directory=remote_job_directory,
    conda_env="/n/groups/datta/tim_sainburg/conda_envs/mmdeploy",
    o2_username=recording_row.username,
    o2_server="login.o2.rc.hms.harvard.edu",
    job_params=params,
    o2_n_cpus=config["o2"]["prediction_2d"]["o2_n_cpus"],
    o2_memory=config["o2"]["prediction_2d"]["o2_memory"],
    o2_time_limit=duration_requested,
    o2_queue=config["o2"]["prediction_2d"]["o2_queue"],
    o2_exclude=config["o2"]["prediction_2d"]["o2_exclude"],
    o2_qos = config["o2"]["prediction_2d"]["o2_qos"] ,
    o2_gres = config["o2"]["prediction_2d"]["o2_gres"] ,
    do_not_submit=True
)

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4)
INFO:paramiko.transport:Auth banner: b'Problems logging in?\nUse your lower case HMS ID, like abc123, not ABC123.\nIf locked out, see:\nhttps://it.hms.harvard.edu/i-want/reset-password-or-unlock-your-hms-account\n'
INFO:paramiko.transport:Authentication (publickey) successful!


In [17]:
if config["prediction_2d"]["use_tensorrt"]:
    runner.python_script = textwrap.dedent(
        f"""
    # load params
    import yaml
    params_file = "{runner.remote_job_directory / f"{runner.job_name}.params.yaml"}"
    config_file = "{config_file.as_posix()}"

    params = yaml.safe_load(open(params_file, 'r'))
    config = yaml.safe_load(open(config_file, 'r'))

    # covert models to tensorrt
    from multicamera_airflow_pipeline.tim_240731.keypoints.tensorrt import RTMModelConverter
    model_converter = RTMModelConverter(
        tensorrt_output_directory = params["tensorrt_model_directory"],
        **config["tensorrt_conversion"]
    )
    model_converter.run()
    
    # run predictions
    from multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D import Inferencer2D
    camera_calibrator = Inferencer2D(
        recording_directory = params["recording_directory"],
        output_directory_predictions = params["output_directory_predictions"],
        expected_video_length_frames = params["expected_video_length_frames"],
        tensorrt_model_directory = params["tensorrt_model_directory"],
        **config["prediction_2d"]
    )
    camera_calibrator.run()
    """
    )
else:
    runner.python_script = textwrap.dedent(
        f"""
    # load params
    import yaml
    params_file = "{runner.remote_job_directory / f"{runner.job_name}.params.yaml"}"
    config_file = "{config_file.as_posix()}"

    params = yaml.safe_load(open(params_file, 'r'))
    config = yaml.safe_load(open(config_file, 'r'))

    # grab sync cameras function
    from multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D import Inferencer2D
    camera_calibrator = Inferencer2D(
        recording_directory = params["recording_directory"],
        output_directory_predictions = params["output_directory_predictions"],
        expected_video_length_frames = params["expected_video_length_frames"],
        tensorrt_model_directory = params["tensorrt_model_directory"],
        **config["prediction_2d"]
    )
    camera_calibrator.run()
    """
    )

print(runner.python_script)


# load params
import yaml
params_file = "/n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240813_102305_477564/24-05-01-13-26-43-110846_2d_predictions_24-08-13-2024-23-05-503582.params.yaml"
config_file = "/n/groups/datta/tim_sainburg/projects/multicamera_airflow_pipeline/multicamera_airflow_pipeline/tim_240731/default_config.yaml"

params = yaml.safe_load(open(params_file, 'r'))
config = yaml.safe_load(open(config_file, 'r'))

# covert models to tensorrt
from multicamera_airflow_pipeline.tim_240731.keypoints.tensorrt import RTMModelConverter
model_converter = RTMModelConverter(
    tensorrt_output_directory = params["tensorrt_model_directory"],
    **config["tensorrt_conversion"]
)
model_converter.run()

# run predictions
from multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D import Inferencer2D
camera_calibrator = Inferencer2D(
    recording_directory = params["recording_directory"],
    output_directory_predictions = params["output_directory_predictions"],
    expect

In [18]:
runner.run()

INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Creating remote job directory: /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240813_102305_477564
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Creating remote directory: /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240813_102305_477564
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Successfully created remote directory: /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240813_102305_477564
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Writing job files to remote directory: /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240813_102305_477564
INFO:paramiko.transport.sftp:[chan 1] Opened sftp connection (server version 3)
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Transferring /tmp/tmpu7wundwg to login.o2.rc.hms.harvard.edu:/n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240813_102305_477564/24-05-01-13-26-43-110846_2d_predictions_24-08-13-2024-23-05-503

In [19]:
# wait until the job is finished
# 10000/60/24 = roughly 1 week
for i in range(10000):
    # check job status every n seconds
    status = runner.check_job_status()
    if status:
        break
    time.sleep(60)

AttributeError: 'O2Runner' object has no attribute 'slurm_job_id'

In [ ]:
# check if sync successfully completed
if check_2d_completion(output_directory_predictions):
    print('2D prediction completed successfully')
else:
    raise ValueError("2D prediction did not complete successfully.")


In [ ]:
!ls /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_102039_489984/

In [ ]:
!tail /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_102039_489984/24-05-01-13-26-43-110846_ephys_sync_24-08-09-2024-20-39-764709.log